# Análise Exploratória dos Dados dos Conglomerados Pruendenciais

O Banco Central do Brasil disponibiliza dados das instituições sob sua supervisão, porém esses dados não são apresentados de maneira clara e objetiva. Embora estejam no formato de valores separados por vírgulas (CSV), eles ainda requerem um tratamento prévio para ser possível extrair informações relevantes de forma eficaz.

Neste notebook, realizaremos uma análise exploratória dos dados referentes aos conglomerados prudenciais e instituições independentes. Um conglomerado prudencial, conforme a Resolução BCB nº 168, de 1º de dezembro de 2021, é definido como um grupo composto por uma instituição de financeira que detém controle, direto ou indireto, sobre outras entidades no Brasil ou no exterior. Essas entidades podem incluir instituições financeiras, outras instituições autorizadas a funcionar pelo Banco Central do Brasil (BCB), instituições de pagamento não autorizadas pelo BCB, entidades que realizam aquisições de operações de crédito ou de direitos creditórios, além de outras pessoas jurídicas que tenham por objetivo exclusivo a participação societária nessas entidades e fundos de investimento. O controle é caracterizado quando a instituição investidora tem poder sobre a entidade controlada, podendo influenciar seus retornos e decisões. A instituição líder do conglomerado é responsável pela consolidação das demonstrações financeiras do grupo, e o Banco Central pode, a seu critério, incluir ou excluir entidades do conglomerado para garantir a correta representação do patrimônio consolidado.

In [1]:
# Importando as bibliotecas necessárias.

# Pandas: manipulação e análise de dados.
import pandas as pd

In [2]:
# Configurando o Pandas para exibir mais dados.
# Exibir todas as colunas (padrão é 20).
pd.set_option('display.max_columns', None)

# Definir largura máxima da coluna (padrão é 50).
pd.set_option('display.max_colwidth', 60)
# Definir precisão de exibição para números flutuantes.
pd.set_option('display.float_format', '{:.2f}'.format)

# Reduzir a quantidade de dados exibidos em DataFrames muito grandes.
pd.set_option('display.large_repr', 'truncate')

In [3]:
# Definindo o caminho do arquivo a ser carregado.
DATA_PATH = '../if_data_processed'
INSTITUTION_TYPE = 'prudential_conglomerates'
REPORT_TYPE = 'summary'
DATA_BASE = '2024-06.csv'
FILE_PATH = f'{DATA_PATH}/{INSTITUTION_TYPE}/{REPORT_TYPE}/{DATA_BASE}'

In [4]:
# Definindo os nomes das colunas do DataFrame, conforme a descrição das colunas.
COLUMN_NAMES = [
    'Instituicao',  # Nome da instituição ou conglomerado no cadastro do Banco Central.
    'Codigo',  # Código da instituição ou conglomerado no cadastro do Banco Central.
    'TipoConsBancario',  # Tipo de Consolidado Bancário.
    'Segmento',  # Segmento conforme Resolução n.º 4.553/2017.
    'TipoConsolidacao',  # Tipo de Consolidação (I) Instituição Independente, (C) Conglomerado.
    'TipoControle',  # Tipo de Controle (1) Público, (2) Privado Nacional, (3) Controle Estrangeiro.
    'Cidade',  # # Cidade da sede da instituição.
    'UF',  # Unidade da Federação onde fica a sede da instituição.
    'DataBase',  # Data-base do relatório.
    'AtivoTotal',  # Ativo Circulante e Realizável a Longo Prazo + Ativo Permanente.
    'CarteiraCredito',  # Carteira de Crédito Classificada.
    'PassivoExigivel',  # Passivo Circulante e Exigível a Longo Prazo e Resultado de Exercícios Futuros.
    'Captacoes',  # Captações de depósitos + Obrigações por Operações Compromissadas 
                  # + Recursos de Aceites Cambiais, Letras Imobiliárias e Hipotecárias,
                  # Debêntures e Similares + Obrigações por Empréstimos e Repasses.
    'PatrimonioLiquido',  # Patrimônio Líquido + Contas de Resultado Credoras
                          # + Contas de Resultado Devedoras. 
    'LucroLiquido',  # Lucro Líquido, excluindo despesas de juros sobre capital.
    'PatrimonioReferencia',  # Montante de capital regulatório formado pela soma
                             # das parcelas de Capital Nível I e Capital Nível II.
    'IndiceBasileia',  # Relação entre o Patrimônio de Referência e Ativos ponderados pelo risco.
    'IndiceImobilizacao',  # Relação entre Ativo Permanente e Patrimônio de Referência.
    'NumAgencias',  # Número de agências incluídas as sedes
    'NumPostosAtendimento'  # Número de postos de atendimento da instituição ou conglomerado
]

In [5]:
# Definindo as descrições dos valores das colunas tipo de consolidado bancário.
descricoes_tipo_consolidado = {
    "b1": "Banco Comercial ou Conglomerado com Carteira Comercial",
    "b2": "Banco Múltiplo sem Carteira Comercial ou Banco de Câmbio/Investimento",
    "b3S": "Cooperativa de Crédito Singular",
    "b3C": "Central e Confederação de Cooperativas de Crédito",
    "b4": "Banco de Desenvolvimento",
    "n1": "Instituição Não Bancária no Mercado de Crédito",
    "n2": "Instituição Não Bancária no Mercado de Capitais",
    "n4": "Instituição de Pagamento"
}

# Definindo as descrições dos valores das colunas tipo de consolidação.
descricoes_tipo_consolidacao = {
    "I": "Instituição Independente",
    "C": "Conglomerado"
}

# Definindo as descrições dos valores das colunas tipo de controle.
descricoes_tipo_controle = {
    1: "Público",
    2: "Privado Nacional",
    3: "Controle Estrangeiro"
}

# Definindo as regiões e estados do Brasil.
regioes_estados = {
    'Norte': ['AC', 'AM', 'AP', 'PA', 'RO', 'RR', 'TO'],  # Ordem alfabética
    'Nordeste': ['AL', 'BA', 'CE', 'MA', 'PB', 'PE', 'PI', 'RN', 'SE'],  # Ordem alfabética
    'Sul': ['PR', 'RS', 'SC'],  # Ordem alfabética
    'Sudeste': ['ES', 'MG', 'RJ', 'SP'],  # Ordem alfabética
    'Centro-oeste': ['DF', 'GO', 'MT', 'MS']  # Ordem alfabética
}
# Mapeando os estados para as regiões do Brasil.
# Formato: {estado: regiao}
estado_para_regiao = {estado: regiao for regiao, estados in regioes_estados.items() for estado in estados}

In [6]:
# Carregando a base de dados para o DataFrame.
# Importante: O arquivo CSV está separado por ponto e vírgula.
df = pd.read_csv(FILE_PATH,
                 sep=';',
                 names=COLUMN_NAMES)

In [7]:
# Primeiro contato com o DataFrame.
df

,Instituicao,Codigo,TipoConsBancario,Segmento,TipoConsolidacao,TipoControle,Cidade,UF,DataBase,AtivoTotal,CarteiraCredito,PassivoExigivel,Captacoes,PatrimonioLiquido,LucroLiquido,PatrimonioReferencia,IndiceBasileia,IndiceImobilizacao,NumAgencias,NumPostosAtendimento
0,ITAU - PRUDENCIAL,1000080099,b1,S1,C,2,SAO PAULO,SP,06/2024,2.636.882.506,946.459.806,2.443.588.602,1.884.371.314,193.293.904,19.963.578,215.556.905,"16,56%","20,86%",2.15,1.10
1,BB - PRUDENCIAL,1000080329,b1,S1,C,1,BRASILIA,DF,06/2024,2.334.367.943,1.024.504.848,2.161.635.722,1.845.389.366,172.732.221,18.173.027,175.348.301,"14,19%","16,35%",4.01,512.00
2,CAIXA ECONÔMICA FEDERAL - PRUDENCIAL,1000080738,b1,S1,C,1,BRASILIA,DF,06/2024,1.907.044.008,1.175.742.235,1.807.735.638,1.617.770.675,99.308.370,5.300.681,129.887.763,"16,40%","9,26%",3.38,846.00
3,BRADESCO - PRUDENCIAL,1000080075,b1,S1,C,2,OSASCO,SP,06/2024,1.682.417.433,658.552.579,1.521.287.541,1.277.833.567,161.129.891,8.937.210,149.119.407,"15,23%","25,57%",2.61,1.05
4,SANTANDER - PRUDENCIAL,1000080185,b1,S1,C,3,SAO PAULO,SP,06/2024,1.279.020.308,540.556.671,1.180.740.535,900.469.651,98.279.773,6.356.898,98.725.721,"14,38%","20,02%",2.53,1.48
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1352,COOPERATIVA CENTRAL DE CRÉDITO COM INTERAÇÃO SOLIDÁRIA -...,10013534,b3C,S4,I,2,LAURO DE FREITAS,BA,06/2024,NI,NI,NI,NI,NI,NI,NI,NI%,NI%,0.00,0.00
1353,COOPERATIVA DE CRÉDITO RURAL DO AGRESTE ALAGOANO - COOPE...,8482873,b3S,S5,I,2,IGACI,AL,06/2024,NI,NI,NI,NI,NI,NI,NI,NI%,NI%,0.00,0.00
1354,COOPERATIVA DE CRÉDITO MÚTUO DOS SERVIDORES MUNICIPAIS D...,5969937,b3S,S5,I,2,BEBEDOURO,SP,06/2024,NI,NI,NI,NI,NI,NI,NI,NI%,NI%,0.00,0.00
1355,COOPERATIVA DE ECONOMIA E CRÉDITO MÚTUO DOS BANCÁRIOS DO...,1301541,b3S,S5,I,2,UBERLANDIA,MG,06/2024,NI,NI,NI,NI,NI,NI,NI,NI%,NI%,0.00,0.00


In [8]:
# Verificando o número de colunas e linhas do DataFrame.
print(f'Número de colunas: {df.shape[1]}')
print(f'Número de linhas: {df.shape[0]}')

Número de colunas: 20
Número de linhas: 1357


In [9]:
# Verificando se o Pandas inferiu corretamente os tipos de dados das colunas.
df.dtypes

Instituicao              object
Codigo                    int64
TipoConsBancario         object
Segmento                 object
TipoConsolidacao         object
TipoControle              int64
Cidade                   object
UF                       object
DataBase                 object
AtivoTotal               object
CarteiraCredito          object
PassivoExigivel          object
Captacoes                object
PatrimonioLiquido        object
LucroLiquido             object
PatrimonioReferencia     object
IndiceBasileia           object
IndiceImobilizacao       object
NumAgencias             float64
NumPostosAtendimento    float64
dtype: object

## Transformação e Limpeza dos Dados

Dados reais nem sempre estão prontos para serem analisados. Eles podem conter erros, valores ausentes, formatação inadequada e outras imperfeições que precisam ser tratadas antes de serem utilizadas. Nesta seção, vamos realizar algunas transformações e limpezas nos dados para garantir que eles estejam prontos para a análise exploratória.

In [10]:
# Removendo a parte ' - PRUDENCIAL' da coluna 'Instituicao'.
df['Instituicao'] = df['Instituicao'].str.replace(' - PRUDENCIAL', '')

In [11]:
# A documentação do Banco Central informa que os dados das colunas
# 'TipoConsBancario', 'TipoConsolidacao' e 'TipoControle' são categóricos.
print(f'Tipos de Consolidado Bancário: {df['TipoConsBancario'].unique().tolist()}')
print(f'Segmento Resolução nº 4.553/2017: {df['TipoConsolidacao'].unique().tolist()}')
print(f'Tipo de Consolidação: {df['TipoControle'].unique().tolist()}')

Tipos de Consolidado Bancário: ['b1', 'b4', 'n1', 'n4', 'b2', 'n2', 'b3C', 'b3S']
Segmento Resolução nº 4.553/2017: ['C', 'I']
Tipo de Consolidação: [2, 1, 3]


In [12]:
# A documentação do Banco Central informa que os dados das colunas
# AtivoTotal, CarteiraCredito, PassivoExigivel, Captacoes, PatrimonioLiquido
# e LucroLiquido são numéricos.
print(f'Exemplo de dados da coluna Ativo Total: {df['AtivoTotal'].loc[0]}')
print(f'Exemplo de dados da coluna Carteira de Crédito Classificada: {df['CarteiraCredito'].loc[0]}')
print(f'Exemplo de dados da coluna Passivo Circulante: {df['PassivoExigivel'].loc[0]}')
print(f'Exemplo de dados da coluna Captações: {df['Captacoes'].loc[0]}')
print(f'Exemplo de dados da coluna Patrimônio Líquido: {df['PatrimonioLiquido'].loc[0]}')
print(f'Exemplo de dados da coluna Lucro Líquido: {df['LucroLiquido'].loc[0]}')

Exemplo de dados da coluna Ativo Total: 2.636.882.506
Exemplo de dados da coluna Carteira de Crédito Classificada: 946.459.806
Exemplo de dados da coluna Passivo Circulante: 2.443.588.602
Exemplo de dados da coluna Captações: 1.884.371.314
Exemplo de dados da coluna Patrimônio Líquido: 193.293.904
Exemplo de dados da coluna Lucro Líquido: 19.963.578


In [13]:
# A documentação do Banco Central informa que os dados das colunas
# 'IndiceBasileia' e 'IndiceImobilizacao' são numéricos, representando um percentual.
print(f'Exemplo de dados da coluna Índice de Basileia: {df['IndiceBasileia'].loc[0]}')
print(f'Exemplo de dados da coluna Índice de Imobilização: {df['IndiceImobilizacao'].loc[0]}')

Exemplo de dados da coluna Índice de Basileia: 16,56%
Exemplo de dados da coluna Índice de Imobilização: 20,86%


In [14]:
def clean_number_format(column: pd.Series) -> pd.Series:
    """
    Limpa a formatação de números em uma coluna do DataFrame.
    
    Parameters:
        column (pandas.Series): A coluna a ser limpa.
    """
    
    # Remover o ponto como separador de milhares e substituir a vírgula por ponto decimal.
    return column.str.replace('.', '', regex=False).str.replace(',', '.', regex=False)

In [15]:
def clean_percentage_format(column: pd.Series) -> pd.Series:
    """
    Limpa a formatação de porcentagens em uma coluna do DataFrame.
    
    Parameters:
        column (pandas.Series): A coluna a ser limpa.
    """
    
    # Remover o símbolo de porcentagem e substituir a vírgula por ponto decimal.
    return column.str.replace('%', '', regex=False).str.replace(',', '.', regex=False)

In [16]:
def change_dtype(data_frame: pd.DataFrame,
                 column: str,
                 new_dtype: str) -> pd.DataFrame:
    """
    Altera o tipo de dados de uma coluna do DataFrame.
    
    Parameters:
        data_frame (pandas.DataFrame): O DataFrame contendo os dados.
        column (str): O nome da coluna a ser alterada.
        new_dtype (type): O novo tipo de dados da coluna.
    
    Returns:
        pandas.DataFrame: O DataFrame com a coluna alterada.
    """

    # Mapeamento de tipos especiais
    special_types = {
        'float': lambda col: pd.to_numeric(clean_number_format(col),
                                           errors='coerce'),
        'int': lambda col: col.astype('int'),
        'datetime': lambda col: pd.to_datetime(col,
                                               errors='coerce'),
        'category': lambda col: col.astype('category'),
        'bool': lambda col: col.astype(bool),
        'str': lambda col: col.astype(str)
    }

    try:
        if new_dtype in special_types and new_dtype == 'float':
            if data_frame[column].dtype == 'object' and data_frame[column].str.contains('%').any():
                data_frame[column] = special_types[new_dtype](clean_percentage_format(data_frame[column]))
            else:
                data_frame[column] = special_types[new_dtype](data_frame[column])
        elif new_dtype in special_types:
            data_frame[column] = special_types[new_dtype](data_frame[column])
        else:
            data_frame[column] = data_frame[column].astype(new_dtype)
    except Exception as error:
        print(f"Erro ao converter a coluna '{column}' para {new_dtype}: {error}")

    return data_frame

In [17]:
# Alterando as colunas com dados categóricos para o tipo 'category'.
df = change_dtype(df, 'TipoConsBancario', 'category')
df = change_dtype(df, 'Segmento', 'category')
df = change_dtype(df, 'TipoConsolidacao', 'category')
df = change_dtype(df, 'TipoControle', 'category')

# Alterando a coluna 'DataBase' para o tipo 'datetime'.
df = change_dtype(df, 'DataBase', 'datetime')

# Alterando as colunas com dados numéricos para os tipos 'float'.
df = change_dtype(df, 'AtivoTotal', 'float')
df = change_dtype(df, 'CarteiraCredito', 'float')
df = change_dtype(df, 'PassivoExigivel', 'float')
df = change_dtype(df, 'Captacoes', 'float')
df = change_dtype(df, 'PatrimonioLiquido', 'float')
df = change_dtype(df, 'LucroLiquido', 'float')
df = change_dtype(df, 'PatrimonioReferencia', 'float')

# Alterando as colunas com dados percentuais para o tipo 'float'.
df = change_dtype(df, 'IndiceBasileia', 'float')
df = change_dtype(df, 'IndiceImobilizacao', 'float')

# Alterando as colunas com dados numéricos para os tipos 'int'.
df = change_dtype(df, 'NumAgencias', 'int')
df = change_dtype(df, 'NumPostosAtendimento', 'int')

In [18]:
# Para facilitar compreensão da dimensionalidade dos dados,
# vamos converter os valores monetários para milhões.

# Função lambda para converter os valores para milhões.
change_to_millions = lambda column: column / 1_000

# Aplicando a função lambda para as colunas de valores monetários.
df['AtivoTotal'] = df['AtivoTotal'].apply(change_to_millions)
df['CarteiraCredito'] = df['CarteiraCredito'].apply(change_to_millions)
df['PassivoExigivel'] = df['PassivoExigivel'].apply(change_to_millions)
df['PatrimonioLiquido'] = df['PatrimonioLiquido'].apply(change_to_millions)
df['LucroLiquido'] = df['LucroLiquido'].apply(change_to_millions)

In [19]:
# Alterando as informações da coluna Cidade para que todas as palavras comecem com letra maiúscula.
df['Cidade'] = df['Cidade'].str.title()

In [20]:
# Criando uma nova coluna 'Regiao' com base na coluna 'UF'.
# Obtendo a posição da coluna 'Estado'.
posicao_estado = df.columns.get_loc('UF')
df.insert(posicao_estado, 'Regiao', df['UF'].map(estado_para_regiao))
# Alterando o tipo de dados da coluna 'Regiao' para 'category'.
df = change_dtype(df, 'Regiao', 'category')

In [21]:
# Identificando as colunas que possuem valores 'NI (Não Informado)'.
df.eq('NI').sum().sort_values(ascending=False).rename('NI Count')

Instituicao             0
Codigo                  0
TipoConsBancario        0
Segmento                0
TipoConsolidacao        0
TipoControle            0
Cidade                  0
Regiao                  0
UF                      0
DataBase                0
AtivoTotal              0
CarteiraCredito         0
PassivoExigivel         0
Captacoes               0
PatrimonioLiquido       0
LucroLiquido            0
PatrimonioReferencia    0
IndiceBasileia          0
IndiceImobilizacao      0
NumAgencias             0
NumPostosAtendimento    0
Name: NI Count, dtype: int64

In [22]:
# Verificando a existência de valores nulos no DataFrame.
df.isnull().sum().sort_values(ascending=False).rename('Null Count')

IndiceImobilizacao      134
PatrimonioReferencia     39
IndiceBasileia           39
CarteiraCredito          29
Captacoes                29
PatrimonioLiquido        29
PassivoExigivel          29
AtivoTotal               29
LucroLiquido             29
Segmento                  2
TipoConsBancario          0
Codigo                    0
Instituicao               0
TipoConsolidacao          0
DataBase                  0
Regiao                    0
UF                        0
TipoControle              0
Cidade                    0
NumAgencias               0
NumPostosAtendimento      0
Name: Null Count, dtype: int64

In [23]:
# Filtrando apenas as linhas que possuem 'NaN' como valor.
df[df.isna().any(axis=1)]

,Instituicao,Codigo,TipoConsBancario,Segmento,TipoConsolidacao,TipoControle,Cidade,Regiao,UF,DataBase,AtivoTotal,CarteiraCredito,PassivoExigivel,Captacoes,PatrimonioLiquido,LucroLiquido,PatrimonioReferencia,IndiceBasileia,IndiceImobilizacao,NumAgencias,NumPostosAtendimento
9,NU PAGAMENTOS,1000084693,n1,S3,C,2,Sao Paulo,Sudeste,SP,2024-06-01,214500.65,101837.59,200136.12,134684439.00,14364.53,4069.33,16574240.00,1581.00,NaN,8,0
16,CIELO IP,1000084710,n4,S5,C,2,Barueri,Sudeste,SP,2024-06-01,100538.99,888.42,80147.54,3667690.00,20391.46,1038.62,9116259.00,3454.00,NaN,1,0
22,PAGSEGURO,1000084813,b1,S4,C,2,Sao Paulo,Sudeste,SP,2024-06-01,66937.97,2460.70,55598.37,40615403.00,11339.60,774.18,9491445.00,3336.00,NaN,6,0
24,BANCO MASTER,1000080367,b1,S3,C,2,Rio De Janeiro,Sudeste,RJ,2024-06-01,57152.26,27542.99,52550.65,49522848.00,4601.60,433.03,NaN,NaN,NaN,16,5
27,MERCADO PAGO IP,1000084820,n1,S4,C,2,Osasco,Sudeste,SP,2024-06-01,46794.94,10188.45,43786.93,24492888.00,3008.01,349.11,2626620.00,1451.00,NaN,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1352,COOPERATIVA CENTRAL DE CRÉDITO COM INTERAÇÃO SOLIDÁRIA -...,10013534,b3C,S4,I,2,Lauro De Freitas,Nordeste,BA,2024-06-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
1353,COOPERATIVA DE CRÉDITO RURAL DO AGRESTE ALAGOANO - COOPE...,8482873,b3S,S5,I,2,Igaci,Nordeste,AL,2024-06-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
1354,COOPERATIVA DE CRÉDITO MÚTUO DOS SERVIDORES MUNICIPAIS D...,5969937,b3S,S5,I,2,Bebedouro,Sudeste,SP,2024-06-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
1355,COOPERATIVA DE ECONOMIA E CRÉDITO MÚTUO DOS BANCÁRIOS DO...,1301541,b3S,S5,I,2,Uberlandia,Sudeste,MG,2024-06-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0


In [24]:
# Substituindo os valores 'NaN' por NA (Not Available).
df = df.fillna(pd.NA)

In [25]:
# Verificando como ficou o DataFrame após as transformações.
df

,Instituicao,Codigo,TipoConsBancario,Segmento,TipoConsolidacao,TipoControle,Cidade,Regiao,UF,DataBase,AtivoTotal,CarteiraCredito,PassivoExigivel,Captacoes,PatrimonioLiquido,LucroLiquido,PatrimonioReferencia,IndiceBasileia,IndiceImobilizacao,NumAgencias,NumPostosAtendimento
0,ITAU,1000080099,b1,S1,C,2,Sao Paulo,Sudeste,SP,2024-06-01,2636882.51,946459.81,2443588.60,1884371314.00,193293.90,19963.58,215556905.00,1656.00,2086.00,2,1
1,BB,1000080329,b1,S1,C,1,Brasilia,Centro-oeste,DF,2024-06-01,2334367.94,1024504.85,2161635.72,1845389366.00,172732.22,18173.03,175348301.00,1419.00,1635.00,4,512
2,CAIXA ECONÔMICA FEDERAL,1000080738,b1,S1,C,1,Brasilia,Centro-oeste,DF,2024-06-01,1907044.01,1175742.24,1807735.64,1617770675.00,99308.37,5300.68,129887763.00,1640.00,926.00,3,846
3,BRADESCO,1000080075,b1,S1,C,2,Osasco,Sudeste,SP,2024-06-01,1682417.43,658552.58,1521287.54,1277833567.00,161129.89,8937.21,149119407.00,1523.00,2557.00,2,1
4,SANTANDER,1000080185,b1,S1,C,3,Sao Paulo,Sudeste,SP,2024-06-01,1279020.31,540556.67,1180740.53,900469651.00,98279.77,6356.90,98725721.00,1438.00,2002.00,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1352,COOPERATIVA CENTRAL DE CRÉDITO COM INTERAÇÃO SOLIDÁRIA -...,10013534,b3C,S4,I,2,Lauro De Freitas,Nordeste,BA,2024-06-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
1353,COOPERATIVA DE CRÉDITO RURAL DO AGRESTE ALAGOANO - COOPE...,8482873,b3S,S5,I,2,Igaci,Nordeste,AL,2024-06-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
1354,COOPERATIVA DE CRÉDITO MÚTUO DOS SERVIDORES MUNICIPAIS D...,5969937,b3S,S5,I,2,Bebedouro,Sudeste,SP,2024-06-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
1355,COOPERATIVA DE ECONOMIA E CRÉDITO MÚTUO DOS BANCÁRIOS DO...,1301541,b3S,S5,I,2,Uberlandia,Sudeste,MG,2024-06-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0


## Análise Exploratória dos Dados

Após a limpeza e transformação dos dados, podemos realizar uma análise exploratória para obter insights sobre o setor financeiro no Brasil. Vamos começar com uma visão geral dos dados, incluindo o número de instituições financeiras, o tipo de consolidado bancário, o segmento, o tipo de consolidação e o tipo de controle.

In [26]:
def group_and_describe_column(data_frame: pd.DataFrame,
                              column: str,
                              descriptions: dict) -> pd.Series:
    """
    Agrupa e descreve os valores de uma coluna do DataFrame,
    aplicando substituições para torná-los mais legíveis.

    Parameters:
        data_frame (pandas.DataFrame): O DataFrame contendo os dados.
        column (str): O nome da coluna a ser agrupada e descrita.
        descriptions (dict): Um dicionário onde as chaves são os
                             valores originais da coluna e os valores
                             são as descrições legíveis.
    """

    return data_frame[column].replace(descriptions).groupby(
        data_frame[column].replace(descriptions),
                                   observed=True).size().sort_values(ascending=False)

In [27]:
group_and_describe_column(df, 'TipoConsBancario', descricoes_tipo_consolidado)

C:\Users\alexc\AppData\Local\Temp\ipykernel_27100\875438165.py:16: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  return data_frame[column].replace(descriptions).groupby(
C:\Users\alexc\AppData\Local\Temp\ipykernel_27100\875438165.py:17: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  data_frame[column].replace(descriptions),


TipoConsBancario
Cooperativa de Crédito Singular                                          760
Instituição Não Bancária no Mercado de Crédito                           218
Instituição Não Bancária no Mercado de Capitais                          132
Banco Comercial ou Conglomerado com Carteira Comercial                    95
Instituição de Pagamento                                                  83
Banco Múltiplo sem Carteira Comercial ou Banco de Câmbio/Investimento     33
Central e Confederação de Cooperativas de Crédito                         32
Banco de Desenvolvimento                                                   4
Name: TipoConsBancario, dtype: int64

A [Resolução nº 4.553/2017](https://normativos.bcb.gov.br/Lists/Normativos/Attachments/50335/Res_4553_v3_L.pdf) do Banco Central do Brasil define os segmentos para fins da aplicação proporcional da regulação prudencial, considerando o porte e a atividade internacional das instituições. Os segmentos são classificados da seguinte forma:
- S1: Bancos múltiplos, bancos comerciais, bancos de investimento, bancos de câmbio e caixas econômicas que (i) tenham porte (Exposição/Produto Interno Bruto) superior a 10%; ou (ii) exerçam atividade internacional relevante (ativos no exterior superiores a US$ 10 bilhões).
- S2: Composto por: (i) bancos múltiplos, bancos comerciais, bancos de investimento, bancos de câmbio e caixas econômicas de porte inferior a 10% e igual ou superior a 1%; e (ii) demais instituições autorizadas a funcionar pelo Banco Central do Brasil de porte igual ou superior a 1% do PIB.
- S3: Instituições de porte inferior a 1% e igual ou superior a 0,1%.
• S4: Instituições de porte inferior a 0,1%.
- S5: Composto por: (i) instituições de porte inferior a 0,1% que utilizem metodologia facultativa simplificada para apuração dos requerimentos mínimos de Patrimônio de Referência (PR), de Nível I e de Capital Principal, exceto bancos múltiplos, bancos comerciais, bancos de investimento, bancos de câmbio e caixas econômicas; e (ii) não sujeitas a apuração de PR.

In [28]:
df.groupby('Segmento').size().sort_values(ascending=False)

C:\Users\alexc\AppData\Local\Temp\ipykernel_27100\3989073230.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby('Segmento').size().sort_values(ascending=False)


Segmento
S5    912
S4    372
S3     56
S2      8
S1      7
dtype: int64

A maioria das instituições financeiras no Brasil pertence ao segmento S5, que inclui bancos de menor porte e outras entidades autorizadas a operar pelo Banco Central do Brasil, com participação inferior a 0,1% do Produto Interno Bruto (PIB). Analisar apenas essa informação poderia levar à conclusão de que o setor financeiro é diversificado, com uma ampla gama de instituições atendendo a diferentes perfis e necessidades de clientes. No entanto, se considerarmos outras variáveis, como o ativo total e o lucro líquido, uma visão mais detalhada sobre a concentração do setor emerge, revelando o verdadeiro grau de dominância das grandes instituições.

In [29]:
group_and_describe_column(df, 'TipoConsolidacao', descricoes_tipo_consolidacao)

C:\Users\alexc\AppData\Local\Temp\ipykernel_27100\875438165.py:16: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  return data_frame[column].replace(descriptions).groupby(
C:\Users\alexc\AppData\Local\Temp\ipykernel_27100\875438165.py:17: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  data_frame[column].replace(descriptions),


TipoConsolidacao
Instituição Independente    1209
Conglomerado                 148
Name: TipoConsolidacao, dtype: int64

In [30]:
group_and_describe_column(df, 'TipoControle', descricoes_tipo_controle)

C:\Users\alexc\AppData\Local\Temp\ipykernel_27100\875438165.py:16: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  return data_frame[column].replace(descriptions).groupby(
C:\Users\alexc\AppData\Local\Temp\ipykernel_27100\875438165.py:17: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  data_frame[column].replace(descriptions),


TipoControle
Privado Nacional        1247
Controle Estrangeiro      81
Público                   29
Name: TipoControle, dtype: int64

In [31]:
df.groupby('Regiao').size().sort_values(ascending=False)

C:\Users\alexc\AppData\Local\Temp\ipykernel_27100\2180569419.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby('Regiao').size().sort_values(ascending=False)


Regiao
Sudeste         779
Sul             364
Centro-oeste     94
Nordeste         81
Norte            39
dtype: int64

In [32]:
df.groupby('UF').size().sort_values(ascending=False)

UF
SP    474
MG    201
RS    138
SC    114
PR    112
RJ     80
GO     39
ES     24
BA     23
MT     23
DF     19
RO     17
MS     13
PA     12
PB     12
CE     12
PE     10
AL      6
MA      6
SE      4
RN      4
PI      4
AM      3
RR      2
AC      2
TO      2
AP      1
dtype: int64

In [33]:
df.groupby(['Regiao', 'UF']).size().sort_values(ascending=False).head(20) 

C:\Users\alexc\AppData\Local\Temp\ipykernel_27100\3376383296.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby(['Regiao', 'UF']).size().sort_values(ascending=False).head(20)


Regiao        UF
Sudeste       SP    474
              MG    201
Sul           RS    138
              SC    114
              PR    112
Sudeste       RJ     80
Centro-oeste  GO     39
Sudeste       ES     24
Nordeste      BA     23
Centro-oeste  MT     23
              DF     19
Norte         RO     17
Centro-oeste  MS     13
Nordeste      CE     12
              PB     12
Norte         PA     12
Nordeste      PE     10
              MA      6
              AL      6
              PI      4
dtype: int64

In [34]:
df.groupby('Cidade').size().sort_values(ascending=False).head(20)

Cidade
Sao Paulo                296
Rio De Janeiro            65
Porto Alegre              50
Belo Horizonte            48
Curitiba                  36
Barueri                   25
Brasilia                  19
Florianopolis             19
Goiania                   17
Vitoria                   13
Fortaleza                 12
Belem                     10
Salvador                  10
Sao Bernardo Do Campo      9
Cuiaba                     9
Joao Pessoa                8
Osasco                     8
Campo Grande               7
Maringa                    7
Uberlandia                 7
dtype: int64

O sistema financeiro nacional é fortemente concentrado na região sudeste do país, com São Paulo liderando em número de instituições financeiras, seguido por Minas Gerais e Rio de Janeiro, sendo que o Rio de Janeiro ocupa a sexta posição. Essa concentração geográfica pode impactar diretamente a concorrência e a diversificação de produtos e serviços financeiros, limitando o acesso a diferentes opções em outras regiões do Brasil.

## Análise Explanatória
Após obtermos uma compreensão inicial dos dados, podemos avançar para a análise. Neste contexto, nosso objetivo será responder a algumas perguntas:
- Quais são as instituições líderes em termos de ativos totais?
- Quais são as instituições que possuem o maior volume de operações de crédito?
- Quais são as instituições que detêm o maior volume de depósitos?
- Quais são as instituições que possuem o maior patrimônio líquido?
- Quais são as instituições com o maior número de agências operando no país?

### Quais são as instituições líderes em termos de ativos totais?

A análise dos ativos totais permite identificar quais instituições financeiras possuem maior controle sobre os bens e direitos que geram benefícios econômicos futuros. Compreender quais instituições lideram neste quesito nos ajuda a avaliar a robustez financeira e a capacidade de investimento de cada uma delas. O foco desta análise será classificar e comparar as instituições com base em seus ativos, destacando aquelas que detêm maior relevância no mercado financeiro.

In [35]:
def filter_and_sort_df(data_frame: pd.DataFrame,
                       filter: pd.Series | None = None,
                       columns: list | None = None,
                       sorting: str | None = None,
                       top_n: int = 10) -> pd.DataFrame:
    """
    Filtra, seleciona colunas e ordena um DataFrame.

    Parameters:
        data_frame (pandas.DataFrame): O DataFrame contendo os dados.
        filter (pd.Series): Lista de condições para filtrar o DataFrame.
        columns (list): Lista de colunas a serem selecionadas.
        sorting (str): Coluna para ordenar o DataFrame.
        top_n (int): Número de linhas a serem retornadas.

    Returns:
        pandas.DataFrame: O DataFrame filtrado, com as colunas selecionadas,
                          ordenado e com as 'top_n' linhas.
    """

    if filter is not None:
        data_frame_filtered = data_frame[filter]
    else:
        data_frame_filtered = df

    return data_frame_filtered[columns].sort_values(by=sorting, ascending=False).head(top_n).reset_index(drop=True)

In [65]:
# Analise do mercado financeiro como um todo.
filter_and_sort_df(df,
                   columns=['UF', 'Instituicao', 'AtivoTotal'],
                   sorting='AtivoTotal')

,UF,Instituicao,AtivoTotal
0,SP,ITAU,2636882.51
1,DF,BB,2334367.94
2,DF,CAIXA ECONÔMICA FEDERAL,1907044.01
3,SP,BRADESCO,1682417.43
4,SP,SANTANDER,1279020.31
5,RJ,BNDES,778001.36
6,RJ,BTG PACTUAL,582263.55
7,SP,SAFRA,294923.88
8,RJ,XP,229605.27
9,SP,NU PAGAMENTOS,214500.65


In [76]:
# Considerando apenas as instituições classificadas como S1.
filter_and_sort_df(df,
                   filter=(df['Segmento'] == 'S1'),
                   columns=['UF', 'Instituicao', 'AtivoTotal'],
                   sorting='AtivoTotal')

,UF,Instituicao,AtivoTotal
0,SP,ITAU,2636882.51
1,DF,BB,2334367.94
2,DF,CAIXA ECONÔMICA FEDERAL,1907044.01
3,SP,BRADESCO,1682417.43
4,SP,SANTANDER,1279020.31
5,RJ,BTG PACTUAL,582263.55
6,SP,PLUXEE IP,NaN


In [77]:
# Considerando apenas as instituições classificadas como S2.
filter_and_sort_df(df,
                   filter=(df['Segmento'] == 'S2'),
                   columns=['UF', 'Instituicao', 'AtivoTotal'],
                   sorting='AtivoTotal')

,UF,Instituicao,AtivoTotal
0,RJ,BNDES,778001.36
1,SP,SAFRA,294923.88
2,RJ,XP,229605.27
3,SP,CITIBANK,208735.49
4,RS,BCO COOPERATIVO SICREDI,172849.16
5,RS,BANRISUL,136280.73
6,SP,VOTORANTIM,134098.73
7,CE,BCO DO NORDESTE DO BRASIL S.A.,68248.58


In [78]:
# Considerando apenas as instituições classificadas como S3.
filter_and_sort_df(df,
                   filter=(df['Segmento'] == 'S3'),
                   columns=['UF', 'Instituicao', 'AtivoTotal'],
                   sorting='AtivoTotal')

,UF,Instituicao,AtivoTotal
0,SP,NU PAGAMENTOS,214500.65
1,DF,BANCOOB,146026.20
2,SP,JP MORGAN CHASE,140372.57
3,SP,BNP PARIBAS,91682.98
4,SP,BANCO C6,85955.93
5,SP,BCO DAYCOVAL S.A,81686.20
6,SP,ABC-BRASIL,67603.05
7,MG,INTER,65492.38
8,RJ,BANCO MASTER,57152.26
9,SP,BCO RABOBANK INTL BRASIL S.A.,51142.97


In [79]:
# Considerando apenas as instituições classificadas como S4.
filter_and_sort_df(df,
                   filter=(df['Segmento'] == 'S4'),
                   columns=['UF', 'Instituicao', 'AtivoTotal'],
                   sorting='AtivoTotal')

,UF,Instituicao,AtivoTotal
0,SP,PAGSEGURO,66937.97
1,SP,MERCADO PAGO IP,46794.94
2,SP,STONE IP,43277.50
3,SP,MIZUHO,32319.44
4,SP,SOCIETE GENERALE,30537.16
5,SP,TERRA INVESTIMENTOS DTVM,27256.70
6,SP,SCANIA,14919.85
7,SP,STELLANTIS,14685.43
8,SP,BR PARTNERS,12668.12
9,SP,WILL IP,12482.92


In [80]:
# Considerando apenas as instituições classificadas como S5.
filter_and_sort_df(df,
                   filter=(df['Segmento'] == 'S5'),
                   columns=['UF', 'Instituicao', 'AtivoTotal'],
                   sorting='AtivoTotal')

,UF,Instituicao,AtivoTotal
0,SP,CIELO IP,100538.99
1,SP,CLOUDWALK IP,11327.05
2,ES,COOPERATIVA DE CRÉDITO CONEXÃO - SICOOB CONEXÃO,9464.19
3,SP,ALELO IP,8163.18
4,MG,SICOOB CREDICOM - COOPERATIVA DE ECONOMIA E CREDITO MUTU...,7427.81
5,GO,COOPERATIVA DE CRÉDITO DE LIVRE ADMISSÃO CENTRO BRASILEI...,7425.10
6,RO,COOPERATIVA DE CRÉDITO E INVESTIMENTO DO SUDOESTE DA AM...,6972.57
7,SP,PAYPAL DO BRASIL IP,5812.43
8,GO,COOPERATIVA DE CRÉDITO DE LIVRE ADMISSÃO DO SUDOESTE GOIANO,5243.07
9,ES,COOPERATIVA DE CRÉDITO SUL-SERRANA DO ESPÍRITO SANTO,4887.50


In [83]:
# Considerando apenas os banco comerciais ou conglomerado com carteira comercial.
filter_and_sort_df(df,
                   filter=(df['TipoConsBancario'] == 'b1'),
                   columns=['UF', 'Instituicao', 'AtivoTotal'],
                   sorting='AtivoTotal')

,UF,Instituicao,AtivoTotal
0,SP,ITAU,2636882.51
1,DF,BB,2334367.94
2,DF,CAIXA ECONÔMICA FEDERAL,1907044.01
3,SP,BRADESCO,1682417.43
4,SP,SANTANDER,1279020.31
5,RJ,BTG PACTUAL,582263.55
6,SP,SAFRA,294923.88
7,RJ,XP,229605.27
8,SP,CITIBANK,208735.49
9,RS,BCO COOPERATIVO SICREDI,172849.16


In [86]:
# Considerando apenas as cooperativas de crédito singular.
filter_and_sort_df(df,
                   filter=(df['TipoConsBancario'] == 'b3S'),
                   columns=['UF', 'Cidade', 'Instituicao', 'AtivoTotal'],
                   sorting='AtivoTotal')

,UF,Cidade,Instituicao,AtivoTotal
0,SP,Bebedouro,COOPERATIVA DE CREDITO CREDICITRUS,15483.29
1,SC,Blumenau,COOPERATIVA DE CREDITO VALE DO ITAJAI - VIACREDI,13522.38
2,SP,Sertaozinho,SICOOB COCRED COOPERATIVA DE CRÉDITO,12206.65
3,MT,Lucas Do Rio Verde,"COOPERATIVA DE CRÉDITO, POUPANÇA E INVESTIMENTO OURO VER...",11845.52
4,PR,Maringa,"COOPERATIVA DE CRÉDITO, POUPANÇA E INVESTIMENTO DEXIS - ...",11280.02
5,PR,Palotina,"COOPERATIVA DE CRÉDITO, POUPANÇA E INVESTIMENTO VALE DO ...",9589.15
6,ES,Linhares,COOPERATIVA DE CRÉDITO CONEXÃO - SICOOB CONEXÃO,9464.19
7,PR,Medianeira,"COOPERATIVA DE CRÉDITO, POUPANÇA E INVESTIMENTO VANGUARD...",9168.10
8,RS,Nova Petropolis,"COOPERATIVA DE CRÉDITO, POUPANÇA E INVESTIMENTO SICREDI ...",9049.75
9,RS,Carlos Barbosa,COOPERATIVA DE CRÉDITO SICREDI SERRANA RS/ES,8913.10


In [87]:
# Considerando apenas as centrais e confederações de cooperativas de crédito.
filter_and_sort_df(df,
                   filter=(df['TipoConsBancario'] == 'b3C'),
                   columns=['UF', 'Cidade', 'Instituicao', 'AtivoTotal'],
                   sorting='AtivoTotal')

,UF,Cidade,Instituicao,AtivoTotal
0,MG,Belo Horizonte,COOPERATIVA CENTRAL DE CRÉDITO DE MINAS GERAIS LTDA. - S...,22272.45
1,SC,Florianopolis,COOPERATIVA CENTRAL DE CRÉDITO DE SANTA CATARINA E RIO G...,18008.75
2,MG,Belo Horizonte,CENTRAL DAS COOPERATIVAS DE ECONOMIA E CREDITO DO ESTADO...,17048.67
3,SP,Ribeirao Preto,COOPERATIVA CENTRAL DE CRÉDITO DO ESTADO DE SÃO PAULO - ...,13647.49
4,PR,Francisco Beltrao,COOPERATIVA CENTRAL DE CRÉDITO COM INTERAÇÃO SOLIDÁRIA -...,11855.08
5,ES,Vitoria,COOPERATIVA CENTRAL DE CREDITO DO ESPIRITO SANTO - SICOO...,10968.01
6,SC,Blumenau,COOPERATIVA CENTRAL DE CRÉDITO - AILOS,10420.12
7,RS,Porto Alegre,"COOPERATIVA CENTRAL DE CRÉDITO, POUPANÇA E INVESTIMENTO ...",9883.92
8,PR,Curitiba,"COOPERATIVA CENTRAL DE CRÉDITO, POUPANÇA E INVESTIMENTO ...",6833.22
9,GO,Goiania,CENTRAL SICOOB UNI DE COOPERATIVAS DE CRÉDITO,6663.53


In [88]:
# Considerando apenas as instituições com controle público.
filter_and_sort_df(df,
                   filter=(df['TipoControle'] == 1),
                   columns=['UF', 'Instituicao', 'AtivoTotal'],
                   sorting='AtivoTotal')

,UF,Instituicao,AtivoTotal
0,DF,BB,2334367.94
1,DF,CAIXA ECONÔMICA FEDERAL,1907044.01
2,RJ,BNDES,778001.36
3,RS,BANRISUL,136280.73
4,CE,BCO DO NORDESTE DO BRASIL S.A.,68248.58
5,PA,BCO DA AMAZONIA S.A.,51049.09
6,ES,BANESTES,38436.73
7,RS,BD REGIONAL DO EXTREMO SUL,23202.57
8,PA,BCO DO EST. DO PA S.A.,18253.22
9,SE,BCO DO EST. DE SE S.A.,10777.03


In [90]:
# Considerando apenas as instituições com controle estrangeiro.
filter_and_sort_df(df,
                   filter=(df['TipoControle'] == 3),
                   columns=['UF', 'Instituicao', 'AtivoTotal'],
                   sorting='AtivoTotal')

,UF,Instituicao,AtivoTotal
0,SP,SANTANDER,1279020.31
1,SP,CITIBANK,208735.49
2,SP,JP MORGAN CHASE,140372.57
3,SP,BNP PARIBAS,91682.98
4,SP,ABC-BRASIL,67603.05
5,SP,BCO RABOBANK INTL BRASIL S.A.,51142.97
6,SP,VOLKSWAGEN,46126.60
7,SP,BCO MUFG BRASIL S.A.,42658.77
8,SP,BOFA MERRILL LYNCH,41690.08
9,SP,GOLDMAN SACHS,37242.83


### Quais são as instituições que possuem o maior volume de operações de crédito?

As operações de crédito são um dos principais indicadores da atividade financeira de uma instituição. Analisar quais instituições possuem o maior volume de crédito nos permite entender seu alcance no fornecimento de empréstimos e financiamentos, bem como o papel que desempenham na economia. Essa análise busca identificar os líderes no mercado de crédito e examinar como essas operações impactam a performance financeira e a posição competitiva das instituições.

In [92]:
# Analise do mercado financeiro como um todo.
filter_and_sort_df(df,
                   columns=['UF', 'Instituicao', 'CarteiraCredito'],
                   sorting='CarteiraCredito')

,UF,Instituicao,CarteiraCredito
0,DF,CAIXA ECONÔMICA FEDERAL,1175742.24
1,DF,BB,1024504.85
2,SP,ITAU,946459.81
3,SP,BRADESCO,658552.58
4,SP,SANTANDER,540556.67
5,RJ,BNDES,323630.39
6,RJ,BTG PACTUAL,142001.23
7,SP,SAFRA,122675.15
8,SP,NU PAGAMENTOS,101837.59
9,SP,VOTORANTIM,74560.40


In [93]:
# Considerando apenas as instituições classificadas como S1.
filter_and_sort_df(df,
                   filter=(df['Segmento'] == 'S1'),
                   columns=['UF', 'Instituicao', 'CarteiraCredito'],
                   sorting='CarteiraCredito')

,UF,Instituicao,CarteiraCredito
0,DF,CAIXA ECONÔMICA FEDERAL,1175742.24
1,DF,BB,1024504.85
2,SP,ITAU,946459.81
3,SP,BRADESCO,658552.58
4,SP,SANTANDER,540556.67
5,RJ,BTG PACTUAL,142001.23
6,SP,PLUXEE IP,NaN


In [94]:
# Considerando apenas as instituições classificadas como S2.
filter_and_sort_df(df,
                   filter=(df['Segmento'] == 'S2'),
                   columns=['UF', 'Instituicao', 'CarteiraCredito'],
                   sorting='CarteiraCredito')

,UF,Instituicao,CarteiraCredito
0,RJ,BNDES,323630.39
1,SP,SAFRA,122675.15
2,SP,VOTORANTIM,74560.40
3,RS,BANRISUL,54716.96
4,RS,BCO COOPERATIVO SICREDI,31873.44
5,SP,CITIBANK,30006.33
6,RJ,XP,26807.51
7,CE,BCO DO NORDESTE DO BRASIL S.A.,13556.43


In [95]:
# Considerando apenas as instituições classificadas como S3.
filter_and_sort_df(df,
                   filter=(df['Segmento'] == 'S3'),
                   columns=['UF', 'Instituicao', 'CarteiraCredito'],
                   sorting='CarteiraCredito')

,UF,Instituicao,CarteiraCredito
0,SP,NU PAGAMENTOS,101837.59
1,SP,BANCO C6,48219.26
2,SP,BCO DAYCOVAL S.A,47880.61
3,SP,VOLKSWAGEN,39850.60
4,MG,INTER,32595.75
5,SP,BCO RABOBANK INTL BRASIL S.A.,32336.43
6,PA,BCO DA AMAZONIA S.A.,27981.11
7,DF,BANCOOB,27846.17
8,RJ,BANCO MASTER,27542.99
9,PR,BANCO CNH INDUSTRIAL CAPITAL S.A,24951.88


In [96]:
# Considerando apenas as instituições classificadas como S4.
filter_and_sort_df(df,
                   filter=(df['Segmento'] == 'S4'),
                   columns=['UF', 'Instituicao', 'CarteiraCredito'],
                   sorting='CarteiraCredito')

,UF,Instituicao,CarteiraCredito
0,SP,SCANIA,14615.28
1,SP,STELLANTIS,13050.48
2,SP,MERCADO PAGO IP,10188.45
3,SP,WILL IP,9159.24
4,SP,HONDA,7639.98
5,PR,PARANÁ BANCO,7338.14
6,PR,BANCO PACCAR,6504.88
7,MT,"COOPERATIVA DE CRÉDITO, POUPANÇA E INVESTIMENTO OURO VER...",6409.49
8,PR,"COOPERATIVA DE CRÉDITO, POUPANÇA E INVESTIMENTO VALE DO ...",6295.08
9,RS,REALIZE CFI,5808.63


In [97]:
# Considerando apenas as instituições classificadas como S5.
filter_and_sort_df(df,
                   filter=(df['Segmento'] == 'S5'),
                   columns=['UF', 'Instituicao', 'CarteiraCredito'],
                   sorting='CarteiraCredito')

,UF,Instituicao,CarteiraCredito
0,ES,COOPERATIVA DE CRÉDITO CONEXÃO - SICOOB CONEXÃO,5295.37
1,RO,COOPERATIVA DE CRÉDITO E INVESTIMENTO DO SUDOESTE DA AM...,4613.64
2,SP,DM FINANCEIRA CFI,4555.39
3,GO,COOPERATIVA DE CRÉDITO DE LIVRE ADMISSÃO CENTRO BRASILEI...,4490.80
4,MG,SICOOB CREDICOM - COOPERATIVA DE ECONOMIA E CREDITO MUTU...,3801.07
5,GO,COOPERATIVA DE CRÉDITO DE LIVRE ADMISSÃO DO SUDOESTE GOIANO,3361.82
6,ES,COOPERATIVA DE CRÉDITO SUL-SERRANA DO ESPÍRITO SANTO,3232.05
7,MG,COOPERATIVA DE CRÉDITO ARACOOP LTDA. - SICOOB ARACOOP,2939.95
8,SC,COOPERATIVA DE CRÉDITO DE LIVRE ADMISSÃO DE ASSOCIADOS S...,2722.78
9,RO,COOPERATIVA DE CRÉDITO DE LIVRE ADMISSÃO DO CENTRO SUL R...,2713.43


In [98]:
# Considerando apenas os banco comerciais ou conglomerado com carteira comercial.
filter_and_sort_df(df,
                   filter=(df['TipoConsBancario'] == 'b1'),
                   columns=['UF', 'Instituicao', 'CarteiraCredito'],
                   sorting='CarteiraCredito')

,UF,Instituicao,CarteiraCredito
0,DF,CAIXA ECONÔMICA FEDERAL,1175742.24
1,DF,BB,1024504.85
2,SP,ITAU,946459.81
3,SP,BRADESCO,658552.58
4,SP,SANTANDER,540556.67
5,RJ,BTG PACTUAL,142001.23
6,SP,SAFRA,122675.15
7,SP,VOTORANTIM,74560.40
8,RS,BANRISUL,54716.96
9,SP,BANCO C6,48219.26


In [99]:
# Considerando apenas as cooperativas de crédito singular.
filter_and_sort_df(df,
                   filter=(df['TipoConsBancario'] == 'b3S'),
                   columns=['UF', 'Cidade', 'Instituicao', 'CarteiraCredito'],
                   sorting='CarteiraCredito')

,UF,Cidade,Instituicao,CarteiraCredito
0,SC,Blumenau,COOPERATIVA DE CREDITO VALE DO ITAJAI - VIACREDI,8568.77
1,MT,Lucas Do Rio Verde,"COOPERATIVA DE CRÉDITO, POUPANÇA E INVESTIMENTO OURO VER...",6409.49
2,PR,Palotina,"COOPERATIVA DE CRÉDITO, POUPANÇA E INVESTIMENTO VALE DO ...",6295.08
3,SP,Bebedouro,COOPERATIVA DE CREDITO CREDICITRUS,6095.76
4,PR,Maringa,"COOPERATIVA DE CRÉDITO, POUPANÇA E INVESTIMENTO DEXIS - ...",5791.69
5,SC,Chapeco,COOPERATIVA DE CRÉDITO MAXI ALFA DE LIVRE ADMISSÃO DE AS...,5493.79
6,PR,Medianeira,"COOPERATIVA DE CRÉDITO, POUPANÇA E INVESTIMENTO VANGUARD...",5387.08
7,ES,Linhares,COOPERATIVA DE CRÉDITO CONEXÃO - SICOOB CONEXÃO,5295.37
8,SP,Sertaozinho,SICOOB COCRED COOPERATIVA DE CRÉDITO,5194.36
9,RS,Carlos Barbosa,COOPERATIVA DE CRÉDITO SICREDI SERRANA RS/ES,4909.06


In [101]:
# Considerando apenas as instituições com controle público.
filter_and_sort_df(df,
                   filter=(df['TipoControle'] == 1),
                   columns=['UF', 'Instituicao', 'CarteiraCredito'],
                   sorting='CarteiraCredito')

,UF,Instituicao,CarteiraCredito
0,DF,CAIXA ECONÔMICA FEDERAL,1175742.24
1,DF,BB,1024504.85
2,RJ,BNDES,323630.39
3,RS,BANRISUL,54716.96
4,PA,BCO DA AMAZONIA S.A.,27981.11
5,RS,BD REGIONAL DO EXTREMO SUL,19810.09
6,PA,BCO DO EST. DO PA S.A.,13843.00
7,CE,BCO DO NORDESTE DO BRASIL S.A.,13556.43
8,ES,BANESTES,10561.08
9,MG,BCO DES. DE MG S.A.,6182.06


In [102]:
# Considerando apenas as instituições com controle estrangeiro.
filter_and_sort_df(df,
                   filter=(df['TipoControle'] == 3),
                   columns=['UF', 'Instituicao', 'CarteiraCredito'],
                   sorting='CarteiraCredito')

,UF,Instituicao,CarteiraCredito
0,SP,SANTANDER,540556.67
1,SP,VOLKSWAGEN,39850.60
2,SP,BCO RABOBANK INTL BRASIL S.A.,32336.43
3,SP,CITIBANK,30006.33
4,PR,BANCO CNH INDUSTRIAL CAPITAL S.A,24951.88
5,SP,ABC-BRASIL,21979.27
6,SP,BCO CSF S.A.,19425.26
7,SP,MERCEDES-BENZ,19109.64
8,RS,BCO DE LAGE LANDEN BRASIL S.A.,15957.43
9,SP,JOHN DEERE,15819.64


### Quais são as instituições que detêm o maior volume de depósitos?

O volume de depósitos é um indicador fundamental da confiança dos clientes e da liquidez de uma instituição financeira. Através dessa análise, examinaremos quais instituições conseguem captar o maior volume de depósitos e como essa captação reflete sua capacidade de operar com segurança e expandir suas atividades de crédito e investimento. O objetivo é destacar os bancos que mais se destacam na obtenção de recursos via depósitos e explorar as implicações dessa liderança.

In [103]:
# Analise do mercado financeiro como um todo.
filter_and_sort_df(df,
                   columns=['UF', 'Instituicao', 'Captacoes'],
                   sorting='Captacoes')

,UF,Instituicao,Captacoes
0,SP,ITAU,1884371314.00
1,DF,BB,1845389366.00
2,DF,CAIXA ECONÔMICA FEDERAL,1617770675.00
3,SP,BRADESCO,1277833567.00
4,SP,SANTANDER,900469651.00
5,RJ,BTG PACTUAL,400107723.00
6,SP,SAFRA,226663798.00
7,RS,BCO COOPERATIVO SICREDI,150435917.00
8,RJ,XP,149103281.00
9,SP,NU PAGAMENTOS,134684439.00


In [104]:
# Considerando apenas as instituições classificadas como S1.
filter_and_sort_df(df,
                   filter=(df['Segmento'] == 'S1'),
                   columns=['UF', 'Instituicao', 'Captacoes'],
                   sorting='Captacoes')

,UF,Instituicao,Captacoes
0,SP,ITAU,1884371314.00
1,DF,BB,1845389366.00
2,DF,CAIXA ECONÔMICA FEDERAL,1617770675.00
3,SP,BRADESCO,1277833567.00
4,SP,SANTANDER,900469651.00
5,RJ,BTG PACTUAL,400107723.00
6,SP,PLUXEE IP,NaN


In [105]:
# Considerando apenas as instituições classificadas como S2.
filter_and_sort_df(df,
                   filter=(df['Segmento'] == 'S2'),
                   columns=['UF', 'Instituicao', 'Captacoes'],
                   sorting='Captacoes')

,UF,Instituicao,Captacoes
0,SP,SAFRA,226663798.00
1,RS,BCO COOPERATIVO SICREDI,150435917.00
2,RJ,XP,149103281.00
3,SP,CITIBANK,128034247.00
4,RJ,BNDES,127433871.00
5,RS,BANRISUL,113501914.00
6,SP,VOTORANTIM,98369066.00
7,CE,BCO DO NORDESTE DO BRASIL S.A.,22123832.00


In [106]:
# Considerando apenas as instituições classificadas como S3.
filter_and_sort_df(df,
                   filter=(df['Segmento'] == 'S3'),
                   columns=['UF', 'Instituicao', 'Captacoes'],
                   sorting='Captacoes')

,UF,Instituicao,Captacoes
0,SP,NU PAGAMENTOS,134684439.00
1,DF,BANCOOB,121775844.00
2,SP,BCO DAYCOVAL S.A,67349723.00
3,SP,JP MORGAN CHASE,64789200.00
4,SP,BANCO C6,62723431.00
5,RJ,BANCO MASTER,49522848.00
6,MG,INTER,48269043.00
7,SP,ABC-BRASIL,46269917.00
8,SP,BCO RABOBANK INTL BRASIL S.A.,39649876.00
9,SP,ORIGINAL,37818655.00


In [107]:
# Considerando apenas as instituições classificadas como S4.
filter_and_sort_df(df,
                   filter=(df['Segmento'] == 'S4'),
                   columns=['UF', 'Instituicao', 'Captacoes'],
                   sorting='Captacoes')

,UF,Instituicao,Captacoes
0,SP,PAGSEGURO,40615403.00
1,SP,MERCADO PAGO IP,24492888.00
2,SP,MIZUHO,13040681.00
3,SP,SCANIA,13032086.00
4,SP,BR PARTNERS,10628885.00
5,SP,STELLANTIS,10553879.00
6,SP,SICOOB COCRED COOPERATIVA DE CRÉDITO,9488119.00
7,SP,STONE IP,9349059.00
8,SE,BCO DO EST. DE SE S.A.,8883344.00
9,PR,PARANÁ BANCO,8587201.00


In [108]:
# Considerando apenas as instituições classificadas como S5.
filter_and_sort_df(df,
                   filter=(df['Segmento'] == 'S5'),
                   columns=['UF', 'Instituicao', 'Captacoes'],
                   sorting='Captacoes')

,UF,Instituicao,Captacoes
0,ES,COOPERATIVA DE CRÉDITO CONEXÃO - SICOOB CONEXÃO,6361429.00
1,GO,COOPERATIVA DE CRÉDITO DE LIVRE ADMISSÃO CENTRO BRASILEI...,6004636.00
2,MG,SICOOB CREDICOM - COOPERATIVA DE ECONOMIA E CREDITO MUTU...,5860286.00
3,RO,COOPERATIVA DE CRÉDITO E INVESTIMENTO DO SUDOESTE DA AM...,4486962.00
4,PR,COOPERATIVA DE CRÉDITO SICOOB METROPOLITANO,3781195.00
5,SP,CIELO IP,3667690.00
6,ES,COOPERATIVA DE CRÉDITO SUL-SERRANA DO ESPÍRITO SANTO,3566166.00
7,SC,COOPERATIVA DE CRÉDITO DE LIVRE ADMISSÃO DE ASSOCIADOS S...,3535940.00
8,GO,COOPERATIVA DE CRÉDITO DE LIVRE ADMISSÃO DO SUDOESTE GOIANO,3173780.00
9,SC,COOPERATIVA DE CRÉDITO UNICRED VALOR CAPITAL LTDA - UNIC...,3044388.00


In [110]:
# Considerando apenas os banco comerciais ou conglomerado com carteira comercial.
filter_and_sort_df(df,
                   filter=(df['TipoConsBancario'] == 'b1'),
                   columns=['UF', 'Instituicao', 'Captacoes'],
                   sorting='Captacoes')

,UF,Instituicao,Captacoes
0,SP,ITAU,1884371314.00
1,DF,BB,1845389366.00
2,DF,CAIXA ECONÔMICA FEDERAL,1617770675.00
3,SP,BRADESCO,1277833567.00
4,SP,SANTANDER,900469651.00
5,RJ,BTG PACTUAL,400107723.00
6,SP,SAFRA,226663798.00
7,RS,BCO COOPERATIVO SICREDI,150435917.00
8,RJ,XP,149103281.00
9,SP,CITIBANK,128034247.00


In [111]:
# Considerando apenas as cooperativas de crédito singular.
filter_and_sort_df(df,
                   filter=(df['TipoConsBancario'] == 'b3S'),
                   columns=['UF', 'Cidade', 'Instituicao', 'Captacoes'],
                   sorting='Captacoes')

,UF,Cidade,Instituicao,Captacoes
0,SP,Bebedouro,COOPERATIVA DE CREDITO CREDICITRUS,11181297.00
1,SC,Blumenau,COOPERATIVA DE CREDITO VALE DO ITAJAI - VIACREDI,9783533.00
2,SP,Sertaozinho,SICOOB COCRED COOPERATIVA DE CRÉDITO,9488119.00
3,SC,Chapeco,COOPERATIVA DE CRÉDITO MAXI ALFA DE LIVRE ADMISSÃO DE AS...,6726855.00
4,ES,Linhares,COOPERATIVA DE CRÉDITO CONEXÃO - SICOOB CONEXÃO,6361429.00
5,PR,Londrina,SISPRIME DO BRASIL - COOPERATIVA DE CRÉDITO,6316714.00
6,RS,Nova Petropolis,"COOPERATIVA DE CRÉDITO, POUPANÇA E INVESTIMENTO SICREDI ...",6299718.00
7,GO,Goiania,COOPERATIVA DE CRÉDITO DE LIVRE ADMISSÃO CENTRO BRASILEI...,6004636.00
8,PR,Maringa,"COOPERATIVA DE CRÉDITO, POUPANÇA E INVESTIMENTO DEXIS - ...",5892248.00
9,MG,Belo Horizonte,SICOOB CREDICOM - COOPERATIVA DE ECONOMIA E CREDITO MUTU...,5860286.00


In [112]:
# Considerando apenas as centrais e confederações de cooperativas de crédito.
filter_and_sort_df(df,
                   filter=(df['TipoConsBancario'] == 'b3C'),
                   columns=['UF', 'Cidade', 'Instituicao', 'Captacoes'],
                   sorting='Captacoes')

,UF,Cidade,Instituicao,Captacoes
0,PR,Francisco Beltrao,COOPERATIVA CENTRAL DE CRÉDITO COM INTERAÇÃO SOLIDÁRIA -...,7245964.00
1,RS,Passo Fundo,COOPERATIVA CENTRAL DE CRÉDITO E INVESTIMENTO COM INTERA...,2417371.00
2,SC,Chapeco,COOPERATIVA CENTRAL DE CRÉDITO E ECONOMIA COM INTERAÇÃO ...,2195934.00
3,SC,Blumenau,COOPERATIVA CENTRAL DE CRÉDITO - AILOS,2037302.00
4,SP,Sao Paulo,CENTRAL DAS COOPERATIVAS DE CRÉDITO DO ESTADO DE SÃO PAU...,1480366.00
5,SP,Ribeirao Preto,COOPERATIVA CENTRAL DE CRÉDITO DO ESTADO DE SÃO PAULO - ...,978013.00
6,SC,Florianopolis,COOPERATIVA CENTRAL DE CRÉDITO DE SANTA CATARINA E RIO G...,970633.00
7,SC,Florianopolis,CONFEDERAÇÃO NACIONAL DAS COOPERATIVAS CENTRAIS DE CRÉDI...,441919.00
8,MG,Belo Horizonte,COOPERATIVA CENTRAL DE CRÉDITO DE MINAS GERAIS LTDA. - S...,300722.00
9,ES,Vitoria,COOPERATIVA CENTRAL DE CREDITO DO ESPIRITO SANTO - SICOO...,242117.00


In [113]:
# Considerando apenas as instituições com controle público.
filter_and_sort_df(df,
                   filter=(df['TipoControle'] == 1),
                   columns=['UF', 'Instituicao', 'Captacoes'],
                   sorting='Captacoes')

,UF,Instituicao,Captacoes
0,DF,BB,1845389366.00
1,DF,CAIXA ECONÔMICA FEDERAL,1617770675.00
2,RJ,BNDES,127433871.00
3,RS,BANRISUL,113501914.00
4,ES,BANESTES,34641156.00
5,CE,BCO DO NORDESTE DO BRASIL S.A.,22123832.00
6,RS,BD REGIONAL DO EXTREMO SUL,17502028.00
7,PA,BCO DO EST. DO PA S.A.,15459490.00
8,PA,BCO DA AMAZONIA S.A.,14540059.00
9,SE,BCO DO EST. DE SE S.A.,8883344.00


In [114]:
# Considerando apenas as instituições com controle estrangeiro.
filter_and_sort_df(df,
                   filter=(df['TipoControle'] == 3),
                   columns=['UF', 'Instituicao', 'Captacoes'],
                   sorting='Captacoes')

,UF,Instituicao,Captacoes
0,SP,SANTANDER,900469651.00
1,SP,CITIBANK,128034247.00
2,SP,JP MORGAN CHASE,64789200.00
3,SP,ABC-BRASIL,46269917.00
4,SP,BCO RABOBANK INTL BRASIL S.A.,39649876.00
5,SP,VOLKSWAGEN,36375604.00
6,BA,BOCOM,23945846.00
7,SP,BNP PARIBAS,23448009.00
8,SP,BOFA MERRILL LYNCH,21828502.00
9,PR,BANCO CNH INDUSTRIAL CAPITAL S.A,21153937.00


Quais são as instituições que possuem o maior patrimônio líquido?

O patrimônio líquido reflete a solidez financeira de uma instituição, sendo uma medida importante de sua capacidade de enfrentar adversidades econômicas. Nesta análise, avaliaremos quais instituições possuem o maior patrimônio líquido, entendendo como esse indicador se relaciona com sua estabilidade, capacidade de crescimento e resiliência em períodos de crise. A análise buscará também observar as tendências de crescimento do patrimônio ao longo do tempo.

In [115]:
# Analise do mercado financeiro como um todo.
filter_and_sort_df(df,
                   columns=['UF', 'Instituicao', 'PatrimonioLiquido'],
                   sorting='PatrimonioLiquido')

,UF,Instituicao,PatrimonioLiquido
0,SP,ITAU,193293.90
1,DF,BB,172732.22
2,SP,BRADESCO,161129.89
3,RJ,BNDES,160070.04
4,DF,CAIXA ECONÔMICA FEDERAL,99308.37
5,SP,SANTANDER,98279.77
6,RJ,BTG PACTUAL,57766.84
7,SP,SAFRA,27888.49
8,SP,CIELO IP,20391.46
9,RJ,BCO CLASSICO S.A.,15901.28


In [116]:
# Considerando apenas as instituições classificadas como S1.
filter_and_sort_df(df,
                   filter=(df['Segmento'] == 'S1'),
                   columns=['UF', 'Instituicao', 'PatrimonioLiquido'],
                   sorting='PatrimonioLiquido')

,UF,Instituicao,PatrimonioLiquido
0,SP,ITAU,193293.90
1,DF,BB,172732.22
2,SP,BRADESCO,161129.89
3,DF,CAIXA ECONÔMICA FEDERAL,99308.37
4,SP,SANTANDER,98279.77
5,RJ,BTG PACTUAL,57766.84
6,SP,PLUXEE IP,NaN


In [117]:
# Considerando apenas as instituições classificadas como S2.
filter_and_sort_df(df,
                   filter=(df['Segmento'] == 'S2'),
                   columns=['UF', 'Instituicao', 'PatrimonioLiquido'],
                   sorting='PatrimonioLiquido')

,UF,Instituicao,PatrimonioLiquido
0,RJ,BNDES,160070.04
1,SP,SAFRA,27888.49
2,SP,VOTORANTIM,14210.89
3,SP,CITIBANK,12801.44
4,CE,BCO DO NORDESTE DO BRASIL S.A.,11563.87
5,RS,BANRISUL,10099.11
6,RJ,XP,8844.33
7,RS,BCO COOPERATIVO SICREDI,4795.89


In [118]:
# Considerando apenas as instituições classificadas como S3.
filter_and_sort_df(df,
                   filter=(df['Segmento'] == 'S3'),
                   columns=['UF', 'Instituicao', 'PatrimonioLiquido'],
                   sorting='PatrimonioLiquido')

,UF,Instituicao,PatrimonioLiquido
0,RJ,BCO CLASSICO S.A.,15901.28
1,SP,NU PAGAMENTOS,14364.53
2,SP,JP MORGAN CHASE,10523.08
3,DF,APE POUPEX,7662.45
4,MG,INTER,7347.30
5,SP,BCO DAYCOVAL S.A,6761.65
6,PA,BCO DA AMAZONIA S.A.,6461.35
7,SP,CREDIT SUISSE,6406.67
8,SP,ABC-BRASIL,6253.35
9,SP,MORGAN STANLEY,5821.26


In [119]:
# Considerando apenas as instituições classificadas como S4.
filter_and_sort_df(df,
                   filter=(df['Segmento'] == 'S4'),
                   columns=['UF', 'Instituicao', 'PatrimonioLiquido'],
                   sorting='PatrimonioLiquido')

,UF,Instituicao,PatrimonioLiquido
0,SP,STONE IP,12128.35
1,SP,PAGSEGURO,11339.60
2,SP,CREFISA S.A. CFI,6425.64
3,SP,HONDA,5231.43
4,PR,AF PARANÁ,3704.32
5,SP,STELLANTIS,3576.26
6,SP,AF DESENVOLVE SP S.A.,3315.60
7,SP,MERCADO PAGO IP,3008.01
8,SP,RODOBENS,2182.03
9,PR,BANCO PACCAR,1831.19


In [120]:
# Considerando apenas as instituições classificadas como S5.
filter_and_sort_df(df,
                   filter=(df['Segmento'] == 'S5'),
                   columns=['UF', 'Instituicao', 'PatrimonioLiquido'],
                   sorting='PatrimonioLiquido')

,UF,Instituicao,PatrimonioLiquido
0,SP,CIELO IP,20391.46
1,SP,CLOUDWALK IP,5012.03
2,ES,COOPERATIVA DE CRÉDITO CONEXÃO - SICOOB CONEXÃO,1722.94
3,GO,COOPERATIVA DE CRÉDITO DE LIVRE ADMISSÃO DO SUDOESTE GOIANO,1437.53
4,RO,COOPERATIVA DE CRÉDITO E INVESTIMENTO DO SUDOESTE DA AM...,1106.65
5,MG,SICOOB CREDICOM - COOPERATIVA DE ECONOMIA E CREDITO MUTU...,1077.75
6,SP,DM FINANCEIRA CFI,1051.31
7,GO,COOPERATIVA DE CRÉDITO DE LIVRE ADMISSÃO CENTRO BRASILEI...,983.75
8,ES,COOPERATIVA DE CRÉDITO SUL-SERRANA DO ESPÍRITO SANTO,901.11
9,SP,ALELO IP,751.10


In [121]:
# Considerando apenas os banco comerciais ou conglomerado com carteira comercial.
filter_and_sort_df(df,
                   filter=(df['TipoConsBancario'] == 'b1'),
                   columns=['UF', 'Instituicao', 'PatrimonioLiquido'],
                   sorting='PatrimonioLiquido')

,UF,Instituicao,PatrimonioLiquido
0,SP,ITAU,193293.90
1,DF,BB,172732.22
2,SP,BRADESCO,161129.89
3,DF,CAIXA ECONÔMICA FEDERAL,99308.37
4,SP,SANTANDER,98279.77
5,RJ,BTG PACTUAL,57766.84
6,SP,SAFRA,27888.49
7,RJ,BCO CLASSICO S.A.,15901.28
8,SP,VOTORANTIM,14210.89
9,SP,CITIBANK,12801.44


In [122]:
# Considerando apenas as cooperativas de crédito singular.
filter_and_sort_df(df,
                   filter=(df['TipoConsBancario'] == 'b3S'),
                   columns=['UF', 'Cidade', 'Instituicao', 'PatrimonioLiquido'],
                   sorting='PatrimonioLiquido')

,UF,Cidade,Instituicao,PatrimonioLiquido
0,SP,Bebedouro,COOPERATIVA DE CREDITO CREDICITRUS,2908.72
1,SC,Blumenau,COOPERATIVA DE CREDITO VALE DO ITAJAI - VIACREDI,2759.88
2,ES,Linhares,COOPERATIVA DE CRÉDITO CONEXÃO - SICOOB CONEXÃO,1722.94
3,SP,Sertaozinho,SICOOB COCRED COOPERATIVA DE CRÉDITO,1523.01
4,MT,Lucas Do Rio Verde,"COOPERATIVA DE CRÉDITO, POUPANÇA E INVESTIMENTO OURO VER...",1464.79
5,GO,Rio Verde,COOPERATIVA DE CRÉDITO DE LIVRE ADMISSÃO DO SUDOESTE GOIANO,1437.53
6,PR,Campo Mourao,CREDICOAMO CREDITO RURAL COOPERATIVA,1424.48
7,PR,Londrina,SISPRIME DO BRASIL - COOPERATIVA DE CRÉDITO,1218.64
8,PR,Maringa,"COOPERATIVA DE CRÉDITO, POUPANÇA E INVESTIMENTO DEXIS - ...",1195.64
9,RS,Cruz Alta,"COOPERATIVA DE CRÉDITO, POUPANÇA E INVESTIMENTO PLANALTO...",1135.37


In [123]:
# Considerando apenas as centrais e confederações de cooperativas de crédito.
filter_and_sort_df(df,
                   filter=(df['TipoConsBancario'] == 'b3C'),
                   columns=['UF', 'Cidade', 'Instituicao', 'PatrimonioLiquido'],
                   sorting='PatrimonioLiquido')

,UF,Cidade,Instituicao,PatrimonioLiquido
0,RS,Porto Alegre,"COOPERATIVA CENTRAL DE CRÉDITO, POUPANÇA E INVESTIMENTO ...",1687.63
1,PR,Curitiba,"COOPERATIVA CENTRAL DE CRÉDITO, POUPANÇA E INVESTIMENTO ...",1368.55
2,MG,Belo Horizonte,COOPERATIVA CENTRAL DE CRÉDITO DE MINAS GERAIS LTDA. - S...,1156.93
3,SC,Florianopolis,COOPERATIVA CENTRAL DE CRÉDITO DE SANTA CATARINA E RIO G...,1000.28
4,MT,Cuiaba,"COOPERATIVA CENTRAL DE CRÉDITO, POUPANÇA E INVESTIMENTO ...",980.94
5,SC,Blumenau,COOPERATIVA CENTRAL DE CRÉDITO - AILOS,785.75
6,MG,Belo Horizonte,CENTRAL DAS COOPERATIVAS DE ECONOMIA E CREDITO DO ESTADO...,721.58
7,GO,Goiania,"COOPERATIVA CENTRAL DE CRÉDITO DE GOIÁS, DISTRITO FEDERA...",653.19
8,ES,Vitoria,COOPERATIVA CENTRAL DE CREDITO DO ESPIRITO SANTO - SICOO...,606.35
9,SP,Ribeirao Preto,COOPERATIVA CENTRAL DE CRÉDITO DO ESTADO DE SÃO PAULO - ...,590.87


In [124]:
# Considerando apenas as instituições com controle público.
filter_and_sort_df(df,
                   filter=(df['TipoControle'] == 1),
                   columns=['UF', 'Instituicao', 'PatrimonioLiquido'],
                   sorting='PatrimonioLiquido')

,UF,Instituicao,PatrimonioLiquido
0,DF,BB,172732.22
1,RJ,BNDES,160070.04
2,DF,CAIXA ECONÔMICA FEDERAL,99308.37
3,CE,BCO DO NORDESTE DO BRASIL S.A.,11563.87
4,RS,BANRISUL,10099.11
5,PA,BCO DA AMAZONIA S.A.,6461.35
6,RS,BD REGIONAL DO EXTREMO SUL,4408.97
7,PR,AF PARANÁ,3704.32
8,SP,AF DESENVOLVE SP S.A.,3315.60
9,ES,BANESTES,2250.87


In [125]:
# Considerando apenas as instituições com controle estrangeiro.
filter_and_sort_df(df,
                   filter=(df['TipoControle'] == 3),
                   columns=['UF', 'Instituicao', 'PatrimonioLiquido'],
                   sorting='PatrimonioLiquido')

,UF,Instituicao,PatrimonioLiquido
0,SP,SANTANDER,98279.77
1,SP,CITIBANK,12801.44
2,SP,JP MORGAN CHASE,10523.08
3,SP,CREDIT SUISSE,6406.67
4,SP,ABC-BRASIL,6253.35
5,SP,MORGAN STANLEY,5821.26
6,SP,BOFA MERRILL LYNCH,5631.95
7,SP,HONDA,5231.43
8,SP,BCO RABOBANK INTL BRASIL S.A.,5042.92
9,SP,VOLKSWAGEN,4921.44


### Quais são as instituições com o maior número de agências operando no país?

O número de agências é um indicador importante da presença física de uma instituição financeira, refletindo seu alcance territorial e proximidade com os clientes. Neste estudo, analisaremos quais instituições financeiras mantêm a maior rede de agências em operação, observando como essa presença se traduz em captação de clientes e na capacidade de penetração no mercado. Também serão consideradas as diferenças regionais e o impacto da digitalização no número de agências.

In [126]:
# Analise do mercado financeiro como um todo.
filter_and_sort_df(df,
                     columns=['UF', 'Instituicao', 'NumAgencias', 'NumPostosAtendimento'],
                     sorting='NumAgencias')

,UF,Instituicao,NumAgencias,NumPostosAtendimento
0,RS,BANRISUL,502,58
1,CE,BCO DO NORDESTE DO BRASIL S.A.,297,1
2,SP,SAFRA,176,6
3,PA,BCO DO EST. DO PA S.A.,163,34
4,DF,BRB,125,122
5,PA,BCO DA AMAZONIA S.A.,116,0
6,ES,BANESTES,94,60
7,RJ,BTG PACTUAL,69,0
8,SE,BCO DO EST. DE SE S.A.,65,9
9,DF,APE POUPEX,63,0


In [127]:
filter_and_sort_df(df,
                     columns=['UF', 'Instituicao', 'NumAgencias', 'NumPostosAtendimento'],
                     sorting='NumPostosAtendimento')

,UF,Instituicao,NumAgencias,NumPostosAtendimento
0,DF,CAIXA ECONÔMICA FEDERAL,3,846
1,DF,BB,4,512
2,MG,MERCANTIL DO BRASIL,11,257
3,SP,COOPERATIVA DE CREDITO CREDICITRUS,0,129
4,DF,BRB,125,122
5,SC,COOPERATIVA DE CREDITO VALE DO ITAJAI - VIACREDI,0,115
6,PR,"COOPERATIVA DE CRÉDITO, POUPANÇA E INVESTIMENTO DEXIS - ...",0,113
7,GO,COOPERATIVA DE CRÉDITO DE LIVRE ADMISSÃO CENTRO BRASILEI...,0,112
8,SC,COOPERATIVA DE CRÉDITO MAXI ALFA DE LIVRE ADMISSÃO DE AS...,0,101
9,PR,COOPERATIVA DE CRÉDITO SICOOB METROPOLITANO,0,100


In [130]:
# Considerando apenas os banco comerciais ou conglomerado com carteira comercial.
filter_and_sort_df(df,
                   filter=(df['TipoConsBancario'] == 'b1'),
                   columns=['UF', 'Instituicao', 'NumAgencias', 'NumPostosAtendimento'],
                   sorting='NumAgencias')

,UF,Instituicao,NumAgencias,NumPostosAtendimento
0,RS,BANRISUL,502,58
1,CE,BCO DO NORDESTE DO BRASIL S.A.,297,1
2,SP,SAFRA,176,6
3,PA,BCO DO EST. DO PA S.A.,163,34
4,DF,BRB,125,122
5,PA,BCO DA AMAZONIA S.A.,116,0
6,ES,BANESTES,94,60
7,RJ,BTG PACTUAL,69,0
8,SE,BCO DO EST. DE SE S.A.,65,9
9,SP,CITIBANK,57,8


In [134]:
# Considerando apenas as cooperativas de crédito singular.
filter_and_sort_df(df,
                   filter=(df['TipoConsBancario'] == 'b3S'),
                   columns=['UF', 'Cidade', 'Instituicao', 'NumAgencias', 'NumPostosAtendimento'],
                   sorting='NumPostosAtendimento')


,UF,Cidade,Instituicao,NumAgencias,NumPostosAtendimento
0,SP,Bebedouro,COOPERATIVA DE CREDITO CREDICITRUS,0,129
1,SC,Blumenau,COOPERATIVA DE CREDITO VALE DO ITAJAI - VIACREDI,0,115
2,PR,Maringa,"COOPERATIVA DE CRÉDITO, POUPANÇA E INVESTIMENTO DEXIS - ...",0,113
3,GO,Goiania,COOPERATIVA DE CRÉDITO DE LIVRE ADMISSÃO CENTRO BRASILEI...,0,112
4,SC,Chapeco,COOPERATIVA DE CRÉDITO MAXI ALFA DE LIVRE ADMISSÃO DE AS...,0,101
5,PR,Maringa,COOPERATIVA DE CRÉDITO SICOOB METROPOLITANO,0,100
6,MT,Tangara Da Serra,"COOPERATIVA DE CRÉDITO, POUPANÇA E INVESTIMENTO DO SUDOE...",0,95
7,PR,Palotina,"COOPERATIVA DE CRÉDITO, POUPANÇA E INVESTIMENTO VALE DO ...",0,95
8,PR,Medianeira,"COOPERATIVA DE CRÉDITO, POUPANÇA E INVESTIMENTO VANGUARD...",0,92
9,RS,Erechim,"COOPERATIVA DE CRÉDITO, POUPANÇA E INVESTIMENTO UNIÃO DE...",0,70


In [136]:
# Considerando apenas as instituições com controle público.
filter_and_sort_df(df,
                   filter=(df['TipoControle'] == 1),
                   columns=['UF', 'Instituicao', 'NumAgencias', 'NumPostosAtendimento'],
                   sorting='NumAgencias')

,UF,Instituicao,NumAgencias,NumPostosAtendimento
0,RS,BANRISUL,502,58
1,CE,BCO DO NORDESTE DO BRASIL S.A.,297,1
2,PA,BCO DO EST. DO PA S.A.,163,34
3,DF,BRB,125,122
4,PA,BCO DA AMAZONIA S.A.,116,0
5,ES,BANESTES,94,60
6,SE,BCO DO EST. DE SE S.A.,65,9
7,RJ,BNDES,5,0
8,RS,BD REGIONAL DO EXTREMO SUL,4,0
9,DF,BB,4,512


In [137]:
filter_and_sort_df(df,
                   filter=(df['TipoControle'] == 1),
                   columns=['UF', 'Instituicao', 'NumAgencias', 'NumPostosAtendimento'],
                   sorting='NumPostosAtendimento')

,UF,Instituicao,NumAgencias,NumPostosAtendimento
0,DF,CAIXA ECONÔMICA FEDERAL,3,846
1,DF,BB,4,512
2,DF,BRB,125,122
3,ES,BANESTES,94,60
4,RS,BANRISUL,502,58
5,PA,BCO DO EST. DO PA S.A.,163,34
6,SE,BCO DO EST. DE SE S.A.,65,9
7,CE,BCO DO NORDESTE DO BRASIL S.A.,297,1
8,RJ,BNDES,5,0
9,RS,BD REGIONAL DO EXTREMO SUL,4,0


In [135]:
# Considerando apenas as instituições com controle estrangeiro.
filter_and_sort_df(df,
                   filter=(df['TipoControle'] == 3),
                   columns=['UF', 'Instituicao', 'NumAgencias', 'NumPostosAtendimento'],
                   sorting='NumAgencias')

,UF,Instituicao,NumAgencias,NumPostosAtendimento
0,SP,CITIBANK,57,8
1,SP,BCO RABOBANK INTL BRASIL S.A.,19,0
2,SP,CCB,16,1
3,SP,ABC-BRASIL,15,0
4,SP,CREDIT SUISSE,10,0
5,RJ,GLOBAL EXCHANGE CC,9,0
6,SP,JP MORGAN CHASE,9,0
7,BA,BOCOM,8,0
8,SP,GUIDE INVESTIMENTOS,8,0
9,SP,TULLETT PREBON BRASIL S.A. CTVM,7,0
